# 킨텍스 장래개발사업 반영 위한 엑셀 input file 생성
### __Total Running Time : 21.6 ~ 22 sec__

Code file과 Data file('킨텍스-사업지구-존세분화-YYMMDD-YYYY.xlsx' 2025년도부터 2050년도까지) 위치는 달라도 상관 없습니다.

- 시나리오 추가됐을 경우에도 코드 정상구현 됩니다.
- 년도 추가되었을 때(파일개수 기존 6개 -> 7개 이상으로 증가)에도 코드 정상구현 됩니다.
- Data file명 바뀌어도 코드작동은 될텐데, 오름차순으로 그대로 append해서 입력하는 거라 완료 후 년도별로 sort 필요할수도 있음.
    - __필요하시면 Trello : 박현(@parkgiraffe) 태그나 따로 연락해주시면 감사하겠습니다.__

## 필요 라이브러리

In [79]:
import pandas as pd
import datetime
import os
import re
import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook

d_today = datetime.date.today() # 파일명 마지막에 오늘날짜 YYMMDD 로 바꿔주기 위해 필요.

### <span style="color:green">수정해야 할 것 : 전역변수 path_dir</span>
- path_dir = 킨텍스-사업지구-존세분화 엑셀 파일(.xlsx)만 있는 경로 주소가 담긴 변수입니다.
- 다른 파일 혹은 디렉토리가 있는 경우 Compile Error가 발생합니다.
- 각자 사용환경에 따라 경로를 바꿔주세요.
- Permission denied 발생 시, path_dir 내에 있는 .xlsx 파일을 종료해주세요.

### Cell Explanation
- 파일이 있는 디렉토리를 설정해주면, 자동으로 내부 파일을 불러들여 sheet name에 있는 공백을 삭제합니다.(전처리 과정)

In [80]:
# path_dir은 킨텍스-사업지구-존세분화 엑셀 파일만이 있는 경로로 지정합니다.
# 현재는 박현의 Local로 지정되어 있으므로, 각자 사용환경에 따라 경로 바꿔주시면 됩니다. 
path_dir = 'C:/GitRepo1/_LocalRepo(Bigfile)/[KINTEX]Cowork_220328_zonefile'

_filelink = list()

for file in os.listdir(path_dir) :
    _filelink.append(path_dir + '/' + file)

for name in _filelink :
    load_wb = load_workbook(name, data_only=True)
    
    # 가끔 띄어쓰기 잘못해서 sheetname에 공백이 포함되어있는 경우 있음, 해당 공백을 제거한 결과로 sheet명을 자동변경하여 저장.
    for sheetname in load_wb.sheetnames :
        new_ws = load_wb[sheetname]
        new_ws.title = new_ws.title.replace(" ", "")
        
    load_wb.save(name)


### Cell Explanation
- <span style = 'color:red'>해당 code는 2025~2050까지의 6개 원본 xlsx 파일 모두가 sheet명, sheet개수, sheet별 양식이 동일하다는 전제로 작성되었습니다.</span>
   - 공백은 제거되지만, sheet name이 년도별 file마다 다를 경우에는 오류가 발생할 수 있습니다.
<br/>
<br/>
- 원시데이터 중 필요한 sheet는 sheet 이름이 '_YYMMDD'로 끝나는 교통량 데이터입니다.
   - 따라서 해당 sheet만 수집하기 위한 code입니다.
   - 해당 sheet만 수집된 workbook을 openpyxl 라이브러리를 통해 만듭니다.

In [81]:
_sheetnamelist = list()


_2025 = pd.read_excel(_filelink[0], sheet_name=None)

for sname, svalue in _2025.items() :
    if sname[-7:] == '_YYMMDD' :
        _sheetnamelist.append(sname)

wb = Workbook()
wb.remove_sheet(wb['Sheet'])
for DFname in _sheetnamelist :
    wb.create_sheet(DFname)

C:\Users\Hyun\AppData\Local\Temp\ipykernel_13716\3212214052.py:11: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  wb.remove_sheet(wb['Sheet'])


### Cell Explanation
- 원시Data를 Dataframe로 불러들여 미리 메모리에 올려두었습니다.
   - 아래 for문에 두니 read를 쓸데없이 반복적으로 하여 속도가 많이 느려서(4m 20s) 따로 빼두었습니다. -> (0m 0.4s로 감소)

In [82]:
yearorigindata_xlsx = list()

for num_filelink in range (0,len(_filelink)) :
    yearorigindata_xlsx.append(pd.read_excel(_filelink[num_filelink], sheet_name=None))

### Cell Explanation
- 각 Sheet마다 Data를 추가하는 과정입니다.
- 1번째 row에 있는 값들은 년도(2025~2050)에 상관없이 동일하여, 2025년 data로 입력하였습니다.
</br>

- 코드 실행이 완료되면, code file(.ipynb) 이 있는 위치에 'result.csv'가 생성됩니다.

In [ ]:
for sheetname in wb.sheetnames :
    
    # 첫번째 row, 즉, index 추가([FD_pccar_taxi-od_D, 859, 849])
    _row1_index = list()
    for i in range (0,len(_2025[sheetname].columns)) :
        _row1_index.append(_2025[sheetname].iloc[0,:].index[i])
        
    wb[sheetname].append(_row1_index)
    
    # 2번째~ row, 연도별 교통량 data 추가.
    for num_filelink in range (0,len(_filelink)) :
        
        data_collist = list()
        for colnum in range (0, len(yearorigindata_xlsx[num_filelink][sheetname].columns)) :
            data_collist.append(yearorigindata_xlsx[num_filelink][sheetname].iloc[0,:].values[colnum])
        
        wb[sheetname].append(data_collist)
            
        #print("{}{}".format(yearorigindata_xlsx[num_filelink][sheetname].iloc[0,:].values[0], sheetname))
    
wb.save('result.xlsx')

### Cell Explanation
- result.xlsx 파일은 한 file 안에 각 sheet가 들어있는 .xlsx 파일입니다.
- 그러나 필요한 file은 sheet별로 .csv 파일을 따로따로 만드는 것이므로, 이 과정을 수행합니다.
</br>

- 코드 실행이 완료되면, code file(.ipynb) 이 있는 위치에 result.xlsx의 sheet 개수만큼 csv파일이 생성됩니다.
    - (원래 result.xlsx 따로 생성 안하고 workbook에서 바로 sheet별로 csv로 내보내려 했는데, 기본 함수에서 제공하는 sheet별 저장 parameter가 없고 <br/>
    따로 구현하려니 result.xlsx 크기가 작을 때는 이 방식이 더 편해서 택했습니다.)

In [ ]:
_result = pd.read_excel('result.xlsx', sheet_name=None)

for sheetname in _result :
    _result[sheetname].to_csv('{}.csv'.format(sheetname.replace(sheetname[-6:], d_today.strftime('%y%m%d'))), index=False, encoding='utf-8')